In [1]:
from json import load
import pandas as pd

In [17]:
with open("maestro_sample_log.json") as jsonfile:
    data = load(jsonfile)
    
jsonfile.close()

In [18]:
processed = []
for i in data:
    sample = data[i]
    name = sample['name']
    storage_slot = sample['storage_slot']['slot']
    storage_tray = sample['storage_slot']['tray']
    substrate = sample['substrate']
    worklist = sample['worklist'] # need to deal with this somehow
    hotplate = sample['hotplate_slot']['hotplate']
    hoteplate_slot = sample['hotplate_slot']['slot']
    processed.append([name, storage_slot, storage_tray, substrate, worklist, hotplate, hoteplate_slot])

In [4]:
df = pd.DataFrame(processed, columns = ['name', 
                                        'storage_slot', 
                                        'storage_tray', 
                                        'substrate', 
                                        'worklist', 
                                        'hotplate', 
                                        'hoteplate_slot'])
df.head(5)

,name,storage_slot,storage_tray,substrate,worklist,hotplate,hoteplate_slot
0,sample0,A1,Tray2,ITO,[{'details': {'destination': 'SpincoaterLiquid...,Hotplate1,B3
1,sample1,A2,Tray2,ITO,[{'details': {'destination': 'SpincoaterLiquid...,Hotplate1,B4
2,sample10,C1,Tray2,ITO,[{'details': {'destination': 'SpincoaterLiquid...,Hotplate1,D2
3,sample11,C2,Tray2,ITO,[{'details': {'destination': 'SpincoaterLiquid...,Hotplate1,D3
4,sample12,C3,Tray2,ITO,[{'details': {'destination': 'SpincoaterLiquid...,Hotplate1,C3


In [5]:
for step in data['sample27']['worklist']:
    print(step)

{'details': {'destination': 'SpincoaterLiquidhandler', 'source': 'Tray1'}, 'id': 'storage_to_spincoater--307ae4af-87f5-49be-9921-6c788cfbc43c', 'name': 'storage_to_spincoater', 'precedent': None, 'sample': 'sample27', 'start': 5205, 'start_actual': 5206.759172201157, 'finish_actual': 5234.246048688889}
{'details': {'drops': [{'air_gap': True, 'blow_out': True, 'height': 2, 'pre_mix': [3, 50], 'rate': 80, 'reuse_tip': False, 'slow_retract': True, 'slow_travel': False, 'solution': {'molarity': 1.4, 'solutes': 'Xu-Recipe-PSK', 'solvent': 'DMF0.75_DMSO0.25', 'well': {'labware': '15mL_Tray1', 'well': 'A4'}}, 'time': -5, 'touch_tip': True, 'volume': 40}, {'air_gap': True, 'blow_out': False, 'height': 0.5, 'pre_mix': [3, 100], 'rate': 450.0, 'reuse_tip': True, 'slow_retract': True, 'slow_travel': True, 'solution': {'molarity': 0, 'solutes': '', 'solvent': 'MethylAcetate', 'well': {'labware': '15mL_Tray1', 'well': 'C4'}}, 'time': 30.0, 'touch_tip': False, 'volume': 75.0}], 'duration': 112, 'st

In [6]:
def flatten_characterization(worklist):
    result = {}
    for task in worklist:
        if 'details' in task and 'characterization_tasks' in task['details']:
            for image_type in task['details']['characterization_tasks']:
                img_station =  image_type['station']
                if 'exposure_times' in image_type['details']:
                    result['{}_exposure_time'.format(img_station)] = image_type['details']['exposure_times']
                elif 'exposure_time' in image_type['details']:
                    result['{}_exposure_time'.format(img_station)] = image_type['details']['exposure_time']
                if 'num_scans' in image_type['details']:
                    result['{}_num_scan'.format(img_station)] = image_type['details']['num_scans']
                elif 'num_scan' in image_type['details']:
                    result['{}_num_scan'.format(img_station)] = image_type['details']['num_scan']
                result['{}_duration'.format(img_station)] = image_type['duration']
        #         result['{}_name'.format(img_station)] = task['name']
                result['{}_position'.format(img_station)] = image_type['position']
    return result

def flatten_drops(worklist):
    """some drops have two steps, how should we deal with it?"""
    result = {}
    i = 1
    for task in worklist:
        if 'details' in task and 'drops' in task['details']:
            for drop_task in task['details']['drops']:
                result['drop{}_air_gap'.format(i)] = drop_task['air_gap']
                result['drop{}_blow_out'.format(i)] = drop_task['blow_out']
                result['drop{}_height'.format(i)] = drop_task['height']
                result['drop{}_pre_mix'.format(i)] = drop_task['pre_mix']
                result['drop{}_rate'.format(i)] = drop_task['rate']
                result['drop{}_reuse_tip'.format(i)] = drop_task['reuse_tip']
                result['drop{}_slow_retract'.format(i)] = drop_task['slow_retract']
                result['drop{}_slow_travel'.format(i)] = drop_task['slow_travel']
#                 result['drop{}_solution'.format(i)] = task['solution']
                result['drop{}_solution_molarity'.format(i)] = drop_task['solution']['molarity']
                result['drop{}_solution_solutes'.format(i)] = drop_task['solution']['solutes']
                result['drop{}_solution_solvent'.format(i)] = drop_task['solution']['solvent']
                result['drop{}_solution_well_labware'.format(i)] = drop_task['solution']['well']['labware']
                result['drop{}_solution_well'.format(i)] = drop_task['solution']['well']['well']
                result['drop{}_time'.format(i)] = drop_task['time']
                result['drop{}_touch_tip'.format(i)] = drop_task['touch_tip']
                result['drop{}_volume'.format(i)] = drop_task['volume']

            result['drop{}_start_times'.format(i)] = task['details']['start_times']
            result['drop{}_duration'.format(i)] = task['details']['duration']
            result['drop{}_steps'.format(i)] = task['details']['steps']
                
            result['drop{}_id'.format(i)] = task['id']
            result['drop{}_name'.format(i)] = task['name']
            result['drop{}_precedent'.format(i)] = task['precedent']
            result['drop{}_sample'.format(i)] = task['sample']
            result['drop{}_start'.format(i)] = task['start']
            result['drop{}_start_actual'.format(i)] = task['start_actual']
            result['drop{}_liquidhandler_timings'.format(i)] = task['liquidhandler_timings']
            result['drop{}_spincoater_log'.format(i)] = task['spincoater_log']
            result['drop{}_headstart'.format(i)] = task['headstart']
            result['drop{}_finish_actual'.format(i)] = task['finish_actual']
            i += 1
    return result

In [7]:
processed = []
for i in data:
    sample = data[i]
    worklist = sample['worklist'] # need to deal with this somehow
    cleaned_dict = {'name': sample['name'], 
                   'storage_slot':sample['storage_slot']['slot'], 
                   'storage_tray':sample['storage_slot']['tray'], 
                   'substrate':sample['substrate'], 
                   'worklist': worklist, 
                   'hotplate':hotplate, 
                   'hoteplate_slot': sample['hotplate_slot']['slot']}
    
    characterization_img = flatten_characterization(worklist)
    drop_step = flatten_drops(worklist)
    processed.append({**cleaned_dict, **characterization_img, **drop_step})

In [8]:
df = pd.DataFrame(processed)
df.head()

,name,storage_slot,storage_tray,substrate,worklist,hotplate,hoteplate_slot,pl_red_exposure_time,pl_red_num_scan,pl_red_duration,...,drop2_id,drop2_name,drop2_precedent,drop2_sample,drop2_start,drop2_start_actual,drop2_liquidhandler_timings,drop2_spincoater_log,drop2_headstart,drop2_finish_actual
0,sample0,A1,Tray2,ITO,[{'details': {'destination': 'SpincoaterLiquid...,Hotplate1,B3,"[0.1, 5, 20]",1,25.32,...,spincoat-8d283a5e-70c7-4d84-91c2-24bc5d759eeb,spincoat,storage_to_spincoater--37c06443-95fc-4767-8c90...,sample0,1110.0,1109.993300,"{'aspirate_solution0': 19.423821687698364, 'st...","{'time': [0.0, 0.21525883674621582, 0.43478918...",31.8746,1200.848833
1,sample1,A2,Tray2,ITO,[{'details': {'destination': 'SpincoaterLiquid...,Hotplate1,B4,"[0.1, 5, 20]",1,25.32,...,spincoat-3701d849-9bc3-4017-8f78-923d22cac5fa,spincoat,storage_to_spincoater--e0bc9d0d-8d0c-4de9-8e0b...,sample1,1282.0,1281.979949,"{'aspirate_solution0': 19.378880262374878, 'st...","{'time': [0.0, 0.21769404411315918, 0.42097878...",31.8746,1372.605357
2,sample10,C1,Tray2,ITO,[{'details': {'destination': 'SpincoaterLiquid...,Hotplate1,D2,"[0.1, 5, 20]",1,25.32,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,sample11,C2,Tray2,ITO,[{'details': {'destination': 'SpincoaterLiquid...,Hotplate1,D3,"[0.1, 5, 20]",1,25.32,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,sample12,C3,Tray2,ITO,[{'details': {'destination': 'SpincoaterLiquid...,Hotplate1,C3,"[0.1, 5, 20]",1,25.32,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
df['worklist'][0]

[{'details': {'destination': 'SpincoaterLiquidhandler', 'source': 'Tray1'},
  'id': 'storage_to_spincoater--d05dec5f-b992-4646-a283-9b253ed34cd6',
  'name': 'storage_to_spincoater',
  'precedent': None,
  'sample': 'sample0',
  'start': 0,
  'start_actual': 0.009992122650146484,
  'finish_actual': 31.29919719696045},
 {'details': {'drops': [{'air_gap': True,
     'blow_out': True,
     'height': 2,
     'pre_mix': [3, 100],
     'rate': 80,
     'reuse_tip': True,
     'slow_retract': True,
     'slow_travel': False,
     'solution': {'molarity': 0.002,
      'solutes': 'SAMS_HTL',
      'solvent': 'Ethanol',
      'well': {'labware': '15mL_Tray1', 'well': 'B4'}},
     'time': -5,
     'touch_tip': True,
     'volume': 40}],
   'duration': 70,
   'start_times': [5],
   'steps': [{'acceleration': 500.0, 'duration': 30.0, 'rpm': 3000.0}]},
  'id': 'spincoat-3b07bc94-0e24-4b50-95f2-7d8aecb1f4ef',
  'name': 'spincoat',
  'precedent': 'storage_to_spincoater--d05dec5f-b992-4646-a283-9b253ed3

In [10]:
# 'duration' for rest (maybe twice)
# 'duration' for anneal

In [11]:
flatten_drops(data['sample0']['worklist'])

{'drop1_air_gap': True,
 'drop1_blow_out': True,
 'drop1_height': 2,
 'drop1_pre_mix': [3, 100],
 'drop1_rate': 80,
 'drop1_reuse_tip': True,
 'drop1_slow_retract': True,
 'drop1_slow_travel': False,
 'drop1_solution_molarity': 0.002,
 'drop1_solution_solutes': 'SAMS_HTL',
 'drop1_solution_solvent': 'Ethanol',
 'drop1_solution_well_labware': '15mL_Tray1',
 'drop1_solution_well': 'B4',
 'drop1_time': -5,
 'drop1_touch_tip': True,
 'drop1_volume': 40,
 'drop1_start_times': [5],
 'drop1_duration': 70,
 'drop1_steps': [{'acceleration': 500.0, 'duration': 30.0, 'rpm': 3000.0}],
 'drop1_id': 'spincoat-3b07bc94-0e24-4b50-95f2-7d8aecb1f4ef',
 'drop1_name': 'spincoat',
 'drop1_precedent': 'storage_to_spincoater--d05dec5f-b992-4646-a283-9b253ed34cd6',
 'drop1_sample': 'sample0',
 'drop1_start': 33,
 'drop1_start_actual': 32.999756813049316,
 'drop1_liquidhandler_timings': {'aspirate_solution': 23.031286239624023,
  'stage_solution': 24.693915605545044,
  'dispense_solution': 30.78004503250122},
